In [1]:
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub, SubscribeListener
from pubnub.enums import PNStatusCategory

pnconfig = PNConfiguration()
pnconfig.subscribe_key="sub-c-b1cadece-f0fa-11e3-928e-02ee2ddab7fe"
# for twitter: subkey: "sub-c-78806dd4-42a6-11e4-aed8-02ee2ddab7fe", channel: "pubnub-twitter"
# for game: subkey: "sub-c-b1cadece-f0fa-11e3-928e-02ee2ddab7fe", channel: "pubnub-game-state"
pnconfig.ssl = False

pubnub = PubNub(pnconfig)

In [2]:
my_listener = SubscribeListener()
pubnub.add_listener(my_listener)

a = pubnub.subscribe().channels('pubnub-game-state').execute()
my_listener.wait_for_connect()
print('connected')

def sampling(a, b, total):
    data = []
    hashMap = [[] for bucket in range(b)]
    for i in range(0, total):
        result = my_listener.wait_for_message_on('pubnub-game-state')
        hashkey = hash(result.message["player_id"]) % len(hashMap)
        filter_condition = filter_data(result, 'action_type', ['Instant', 'AoE'])
        if hashkey <= a and filter_condition:
            data.append(result.message)
    return data

def filter_data(data, var, condition):
    b = [0] * 10
    hash_data = hash(data.message[var]) % 10
    for i in condition:
        hash_condition = hash(i) % 10
        b[hash_condition] = 1
    if (b[hash_data] == 1):
        return True


connected


In [3]:
from collections import Counter
from itertools import compress, product
def combinations(items):
    return [tuple(compress(items,mask)) for mask in product(*[[0,1]]*len(items))]


def countItemSet(data,min_frequent_item=2, set_item=None):
    
    if (set_item == None):
        features = [key for key in data[0]]
        feature_combinations = combinations(features)
        feature_combinations.sort(key=len)
        feature_combinations.pop(0)
    else :
        feature_combinations = set_item
    
    data_frequent = []
    for feature_combination in feature_combinations:
        temp_data = []
        for dt in data:
            temp = []
            for feature in feature_combination :
                temp.append(dt[feature])

            if ( len(temp) > 0):
                temp_data.append(tuple(temp))

        count_items = Counter(temp_data)
        count_items = {key:val for key, val in count_items.items() if val >= min_frequent_item }
        if len(count_items):
            data_frequent.append( (feature_combination, count_items))

    data_frequent = filter(None, data_frequent)
    return list(data_frequent)

In [112]:
from efficient_apriori import apriori

def countItemSetbyApriori(data,min_supp =0.5,min_conf=0):
    features = [key for key in data[0]]     
    temp_data = []
    for dt in data:
        temp = []
        for feature in features :
            temp.append(str(dt[feature]))

        if ( len(temp) > 0):
            temp_data.append(tuple(temp))
            
    return apriori(list(temp_data),min_supp,min_conf)


In [113]:
countItemSetbyApriori(data,0.1,1)

({1: {('13',): 4,
   ('19',): 4,
   ('27',): 4,
   ('AoE',): 11,
   ('Arcane Explosion',): 5,
   ('DoT',): 9,
   ('Frostbolt',): 4,
   ('Immolate',): 4,
   ('Instant',): 11,
   ('Over 9000',): 4,
   ('Rend',): 5,
   ('Ultima',): 4},
  2: {('AoE', 'Arcane Explosion'): 5,
   ('AoE', 'Ultima'): 4,
   ('DoT', 'Immolate'): 4,
   ('DoT', 'Rend'): 5,
   ('Frostbolt', 'Instant'): 4,
   ('Instant', 'Over 9000'): 4}},
 [{Arcane Explosion} -> {AoE},
  {Ultima} -> {AoE},
  {Immolate} -> {DoT},
  {Rend} -> {DoT},
  {Frostbolt} -> {Instant},
  {Over 9000} -> {Instant}])